In [1]:
!pip install mistralai tqdm psycopg2-binary sqlalchemy python-dotenv pandas termcolor openpyxl black --quiet

%load_ext autoreload
%autoreload 2

from database.matching import find_matches, compare_names
from database.getdata import query_data, get_connection, execute_query
import pandas as pd 
import os
from mistralai import Mistral
from dotenv import load_dotenv
import tqdm

import json
load_dotenv()

True

# CHARGEMENT DES DONNEES FILMS, REALISATEURS, PRODUCTEURS, DISTRIBUTEURS

## LOAD DATASET Dataset5050_CNC Films Agréés 2003-2024.xlsx

In [2]:
# use openpyxl to get list of spreadsheet names 
filename = '../database/data/Dataset5050_CNC Films Agréés 2003-2024.xlsx'
xls = pd.ExcelFile(filename)
sheet_names = xls.sheet_names
print(f"{len(sheet_names)} feuilles trouvés dans le fichier {filename}")
sheet_names = [sheet for sheet in sheet_names if '20' in sheet]
sheet_names

22 feuilles trouvés dans le fichier ../database/data/Dataset5050_CNC Films Agréés 2003-2024.xlsx


['2023',
 '2022',
 '2021',
 '2020',
 '2019',
 '2018',
 '2017',
 '2016',
 '2015',
 '2014',
 '2013',
 '2012',
 '2011',
 '2010',
 '2009',
 '2008',
 '2007',
 '2006',
 '2005',
 '2004',
 '2003']

In [3]:
df = pd.read_excel(filename, sheet_name=sheet_names[0], skiprows=4)
df.head()

,VISA,TITRE,REALISATEUR,PRODUCTEURS,DEVIS,GENRE,EOF,RANG,BONUS PARITE,ASR,PAYANT,CLAIR,SOFICA,CREDIT D'IMPOT,AIDE REGIONALE,NATIONALITE
0,158738,Comte de Monte-Cristo (Le),DELAPORTE Matthieu / DE LA PATELLIÈRE Alexandre,Chapter 2 / Pathé Films,42905442,Fiction,OUI,3,NaN,NaN,C+ Ciné+,M6 W9,NaN,X,x,Fr-100
1,160148,De Gaulle - Partie 2,BAUDRY Antonin,Pathé Films,37768806,Fiction,OUI,3,NaN,NaN,C+ Disney+,TF1,x,X,x,Fr-100
2,159694,De Gaulle - Partie 1,BAUDRY Antonin,Pathé Films,37743465,Fiction,OUI,2,NaN,NaN,C+ Disney+,TF1 TMC,x,X,x,Fr-100
3,115903,Amour ouf (L'),LELLOUCHE Gilles,Trésor Films / Chi-Fou-Mi Productions,35752278,Fiction,OUI,3,NaN,NaN,C+ Netflix,FR2,NaN,X,NaN,Fr-100
4,158907,Emilia Perez,AUDIARD Jacques,Page 114 / Why Not Productions,24991495,Fiction,OUI,4,NaN,avant,C+ Ciné+,FR2,NaN,X,x,Fr-100


### REALISATEURS

In [4]:
all_realisateurs = []

for sheet in tqdm.tqdm(sheet_names, desc="Processing sheets"):
    df = pd.read_excel(filename, sheet_name=sheet, skiprows=4)
    
    realisateurs = [[name.strip() for name in tt.split('/')] for tt in df['REALISATEUR'].fillna("").tolist()]
    realisateurs = [item for sublist in realisateurs for item in sublist]
    
    all_realisateurs.extend(realisateurs)
    
all_realisateurs = list(set(all_realisateurs))
all_realisateurs.sort()
all_realisateurs.remove('')
print(f"{len(all_realisateurs)} réalisateurs trouvés dans le fichier {filename}")

Processing sheets:   0%|          | 0/21 [00:00<?, ?it/s]

Processing sheets: 100%|██████████| 21/21 [00:06<00:00,  3.28it/s]

3498 réalisateurs trouvés dans le fichier ../database/data/Dataset5050_CNC Films Agréés 2003-2024.xlsx


In [9]:
df_realisateurs = query_data("SELECT id, type_de_poste, nom FROM inegalites_cinema.cheffes_postes where type_de_poste='Réalisation'")
df_realisateurs.head()

,id,type_de_poste,nom
0,2306,Réalisation,Hamé
1,2307,Réalisation,Ékoué
2,2308,Réalisation,Adrien Beau
3,2309,Réalisation,Adrien Bellay
4,2310,Réalisation,Ainara Vera


In [12]:
find_matches('Ékoue', df_realisateurs, 'nom') 

,id,type_de_poste,nom,similarity_score


In [13]:
realisateur_to_check = []
realisateurs_matched = [] # ['GABARSKI Sam', 'GARBARSKI Sam', 'GAUD Jean-Paul']

for ii, realisateur in tqdm.tqdm(enumerate(all_realisateurs), desc="Processing realisateurs", total=len(all_realisateurs)):
    matches = find_matches(realisateur, df_realisateurs[ii:], 'nom', threshold=0.90)
    if len(matches) > 1:
        print(f"Realisateur: {realisateur}")
        print(matches)
        print()
        realisateur_to_check.append(realisateur)
    elif len(matches) == 1:
        realisateurs_matched.append(realisateur)
    elif len(matches) == 0:
        df_toadd = pd.DataFrame([{
                 'nom': realisateur,
                'type_de_poste': 'Réalisation'
             }])
        df_toadd.to_sql('cheffes_postes', con=get_connection(), schema='inegalites_cinema', if_exists='append', index=False)
        df_realisateurs = pd.concat([df_realisateurs, df_toadd], ignore_index=True)


Processing realisateurs: 100%|██████████| 3498/3498 [02:33<00:00, 22.75it/s]


In [14]:
df_realisateurs = query_data("SELECT id, nom FROM inegalites_cinema.cheffes_postes where type_de_poste='Réalisation'")

In [15]:
# find duplicated entries

ids_to_delete = []
for realisateur in tqdm.tqdm(df_realisateurs["nom"].to_list(), desc="Processing realisateurs"):
    matches = find_matches(realisateur, df_realisateurs, 'nom', threshold=0.9)
    if len(matches) > 1:
        ids = matches['id'].tolist()
        ids_to_delete.extend(ids[1:])
ids_to_delete = list(set(ids_to_delete))


Processing realisateurs:   0%|          | 0/3671 [00:00<?, ?it/s]

Processing realisateurs: 100%|██████████| 3671/3671 [30:58<00:00,  1.98it/s]


In [16]:
with open('../database/realisateurs_matched.txt', 'w') as f:
    for item in realisateurs_matched:
        f.write("%s\n" % item)
        

In [17]:
realisateur_to_check

[]

In [18]:
len(ids_to_delete)
for ii in tqdm.tqdm(ids_to_delete):
    execute_query(f"DELETE FROM inegalites_cinema.cheffes_postes WHERE id = {ii}")

  0%|          | 0/272 [00:00<?, ?it/s]

100%|██████████| 272/272 [00:03<00:00, 70.25it/s]


In [19]:
df_realisateurs = query_data("SELECT id, type_de_poste, nom FROM inegalites_cinema.cheffes_postes where type_de_poste='Réalisation'")
df_realisateurs.head()

,id,type_de_poste,nom
0,4373,Réalisation,ISMAEL
1,4430,Réalisation,JR
2,4544,Réalisation,Kafka
3,4580,Réalisation,LAM-LE
4,4650,Réalisation,LE Lam


In [20]:
find_matches('José ALCALA', df_realisateurs, 'nom')

,id,type_de_poste,nom,similarity_score
310,2796,Réalisation,ALCALA Jose,0.936364


### PROCESS FILMS DATASET

In [ ]:
df_film = pd.read_excel(filename, sheet_name=sheet_names[0], skiprows=4)
df_film.columns = [col.lower().replace(' ', '_') for col in df_film.columns]
df_film["realisateur"] = [[name.strip() for name in tt.split('/')] for tt in df_film['realisateur'].fillna("").tolist()]
df_film = df_film.explode("realisateur")
df_film.columns = ['n_visa_exploitation', 'nom_originel', 'realisateur', 'producteurs', 'devis', 'genre', 'eof',
       'rang', 'bonus_parite_', 'asr', 'payant', 'clair', 'sofica',
       'credit_dimpot', 'aide_regionale', 'nationalite']
df_film.head()

In [ ]:
df_realisateurs = query_data("select id, nom from inegalites_cinema.realisateur")
df_realisateurs.head(3)

In [ ]:
columns_clean = df_film.columns.tolist()
columns_clean.remove('realisateur')

df_film_clean = pd.DataFrame([], columns = ['n_visa_exploitation', 'nom_originel', 'realisateur', 'producteurs', 'devis', 'genre', 'id_realisateur'])

In [ ]:
realisateur_to_check = []
for sheet in sheet_names:
    df_film = pd.read_excel(filename, sheet_name=sheet, skiprows=4)
    df_film.columns = [col.lower().replace(' ', '_') for col in df_film.columns]
    df_film["realisateur"] = [[name.strip() for name in tt.split('/')] for tt in df_film['realisateur'].fillna("").tolist()]
    df_film = df_film.explode("realisateur")
    df_film.columns = ['n_visa_exploitation', 'nom_originel', 'realisateur', 'producteurs', 'devis', 'genre', 'eof',
       'rang', 'bonus_parite_', 'asr', 'payant', 'clair', 'sofica',
       'credit_dimpot', 'aide_regionale', 'nationalite']
    df_film = df_film[['n_visa_exploitation', 'nom_originel', 'realisateur', 'producteurs', 'devis', 'genre']]
    
    # iterate over df_film, match the realisateur with the database and add the id to the dataframe
    for index, row in tqdm.tqdm(df_film.iterrows(), total=len(df_film), desc=f"Processing {sheet}"):
        realisateur = row['realisateur']
        matches = find_matches(realisateur, df_realisateurs, 'nom', threshold=0.9)
        if len(matches) == 1:
            df_toadd = pd.DataFrame([row])
            df_toadd['id_realisateur'] = matches.iloc[0]['id']
            df_film_clean = pd.concat([df_film_clean, df_toadd], ignore_index=True)
        else:
            print(f"Realisateur: {realisateur}")
            print(matches)
            print()
            realisateur_to_check.append(realisateur)
    
    break

In [ ]:
matches.iloc[0]['id']

## LOAD DATASET Dataset5050_César 1976-2024.xlsx

In [ ]:
# Read both sheets from the Excel file
df_films = pd.read_excel('data/Dataset5050_César 1976-2024.xlsx', sheet_name='Films')
df_personnes = pd.read_excel('data/Dataset5050_César 1976-2024.xlsx', sheet_name='Personnes')


In [ ]:
df_films.head()

In [ ]:
df_personnes.head()

## LOAD DATASET Dataset5050_Cheffes de poste 2023.xlsx - LISTE_FINALE_FILMS_RETENUS.xlsx
